In [26]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import features

In [28]:
ames = pd.read_csv("../data/engineered.csv")
ames.drop("Unnamed: 0", axis=1, inplace=True)

In [29]:
model = LinearRegression()

# Select target and features to regress on
target = ames.LogSalePrice
X = ames[['GrLivArea', 'LotFrontage',
      'MSSubClass', 'BldgType',
      'OverallQual', 'Neighborhood',
      'SaleCondition']]
X

,GrLivArea,LotFrontage,MSSubClass,BldgType,OverallQual,Neighborhood,SaleCondition
0,856,66.0,Dwelling_30,1Fam,6,SWISU,Normal
1,1049,42.0,Dwelling_120,TwnhsE,5,Edwards,Normal
2,1001,60.0,Dwelling_30,1Fam,5,IDOTRR,Normal
3,1039,80.0,Dwelling_70,1Fam,4,OldTown,Normal
4,1665,70.0,Dwelling_60,1Fam,8,NWAmes,Normal
...,...,...,...,...,...,...,...
2572,952,66.0,Dwelling_30,1Fam,6,BrkSide,Normal
2573,1733,74.0,Dwelling_20,1Fam,3,Edwards,Normal
2574,2002,82.0,Dwelling_90,Duplex,5,Crawfor,Normal
2575,1842,66.0,Dwelling_60,1Fam,7,CollgCr,Normal


In [30]:
# Dummify categorical features from all features selected for our model
all_categorical_features = set(features.get_categorical_features())
X_categorical_features = list(all_categorical_features.intersection(set(X.columns)))
X = pd.get_dummies(X, columns=X_categorical_features, prefix=X_categorical_features, drop_first=True)
X

,GrLivArea,LotFrontage,OverallQual,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,...,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker
0,856,66.0,6,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,1049,42.0,5,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1001,60.0,5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1039,80.0,4,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,1665,70.0,8,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2572,952,66.0,6,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2573,1733,74.0,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2574,2002,82.0,5,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2575,1842,66.0,7,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
# Create training set and holdout
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.3, random_state=42)

In [32]:
model.fit(X=X_train, y=y_train)

LinearRegression()

In [33]:
# Score model
print(f"R^2 training set: {model.score(X=X_train, y=y_train)}")
print(f"R^2 holdout set: {model.score(X=X_test, y=y_test)}")

R^2 training set: 0.8697149632982363
R^2 holdout set: 0.8611487115099061
